Install rasa_nlu[spacy], rasa_core, nest_asyncio and ipython 


In [1]:
!pip install nest_asyncio==1.3.3

In [2]:
!pip install rasa_nlu[spacy]  #restart runtime after installation and rerun previous cells.

In [3]:

!pip install rasa_core

In [4]:
pip install -U ipython #restart runtime after installation and rerun previous cells.

Import rasa_core, rasa_nlu spacy and sys



In [5]:
import rasa_core
import rasa_nlu
import spacy

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got

In [6]:
import sys
python = sys.executable

download en_core_web_md, a medium-sized english model trained on written web text.

In [7]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 120.8 MB 1.7 MB/s 

    Linking successful
    /usr/local/lib/python3.7/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.7/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



Declearation of intent and sample words/phrases/sentences which refer to the intention decleared, which will be stored inside nlu.md. In other words, these are sample phrases whcih the chatbot will learn from for each intent.

In [8]:
# Writing various intents with examples to nlu.md file

nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- how far
- hey there
- whats up
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- good by
- later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:thanks
- thanks
- thank you
- appreciated
- gracias


## intent:company_name
- What is the name of your company
- company name
- name of the company

## intent:team_info
- What is the total strength of the team
- total team strength
- HR info required
- team count
- count of team members
- strength of the team

## intent:choosing_team
- 1
- 2
- 3
- 4

## intent:males_info
- What are the total number of males in the team
- total number of males
- Male info required
- Male count
- count of males
- strength of the males

## intent:choosing_males
- 1
- 2
- 3
- 4

## intent:females_info
- What are the total number of females in the team
- total number of females
- female info required
- female count
- count of females
- strength of the females

## intent:choosing_females
- 1
- 2
- 3
- 4

## intent:businessage
- What are the total number of years of Business
- total number of business years
- business age info required
- business age count
- range of business
- duration of business

## intent:choosing_businessage
- 1
- 2
- 3
- 4

## intent:team_success 
- Rate your team success
- what is the success of your team
- team success rate

## intent:choosing_teamsuccess
- 1
- 2
- 3
- 4
- 5

## intent:team_diversity 
- provide diversity in your team
- what is the your team's diversity
- team diversity

## intent:choosing_teamdiversity
- 1
- 2
- 3
- 4
- 5

## intent:team_inclusiveness 
- provide inclusiveness in your team
- what is the your team's inclusiveness
- team inclusiveness

## intent:choosing_teaminclusiveness
- 1
- 2
- 3
- 4
- 5

## intent:team_diversitysuccess
- Rate diversitysuccess of your team
- what is your team's diversitysuccess
- team success rate acheived from diversity

## intent:choosing_teamdiversitysuccess
- 1
- 2
- 3
- 4
- 5

## intent:team_country
- what is the country of your team
- what is your team's country
- provide country name of your team

## intent:iswomenowned
- your business women owned 
- is the business women owned
- does women own the business

## intent:choosing_iswomenowned
- 1
- 2

## intent:team_miniorityowned
- your business minority owned 
- is the business minority owned
- does minority communinity own the business

## intent:choosing_teamminiorityowned
- 1
- 2
- 3
- 4
- 5
- 6
- 7
- 8
- 9
- 10
- 11
- 12

"""


%store nlu_md >nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


Setting up our configuration for our pipeline and policies

In [9]:
# Adding the NLU components to the pipeline in config.yml file
config = """
language: "en_core_web_md"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                     # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms

policies:
- name: "RulePolicy"
  # Confidence threshold for the `core_fallback_action_name` to apply.
  # The action will apply if no other action was predicted with
  # a confidence >= core_fallback_threshold
  core_fallback_threshold: 0.68
  core_fallback_action_name: "action_default_fallback"
  enable_fallback_prediction: True


""" 

%store config >config.yml

Writing 'config' (str) to file 'config.yml'.


# Training the model
Import the load_data and config modules so as to load the training data file and config file respectiely, which would be fed to the Trainer module to train the model.

In [10]:
# Import modules for training
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")
trainer = Trainer(config.load("config.yml"))

# training the nlu
interpreter = trainer.train(training_data)
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_

Testing our model.
The aim is to see how well it can predict intents, this would be then use to determind the appropriate response.

In [11]:
 # Testing the NLU model with an input message

import json
def pprint(o):   
    print(json.dumps(o, indent=2))

pprint(interpreter.parse("x"))

{
  "intent": {
    "name": "choosing_teamminiorityowned",
    "confidence": 0.14577796934300974
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "choosing_teamminiorityowned",
      "confidence": 0.14577796934300974
    },
    {
      "name": "choosing_teamdiversity",
      "confidence": 0.06791547464337422
    },
    {
      "name": "choosing_teamdiversitysuccess",
      "confidence": 0.06787423224094576
    },
    {
      "name": "choosing_teaminclusiveness",
      "confidence": 0.06783734730218523
    },
    {
      "name": "choosing_teamsuccess",
      "confidence": 0.06737350787519648
    },
    {
      "name": "greet",
      "confidence": 0.061218818795391663
    },
    {
      "name": "choosing_females",
      "confidence": 0.048185496455169514
    },
    {
      "name": "choosing_businessage",
      "confidence": 0.04818499542365934
    },
    {
      "name": "team_info",
      "confidence": 0.04811314087370736
    },
    {
      "name": "choosing_males",
     

This is the second part of the project where we would be dealing with stories, actions and templates so our bot can respond to queries appropriately. Stories is a path which countains intents and their respective actions to be taken.
Actions are the the responses we want our bot to give after identifying intent.
Templates are where we store all the actions.
Slots are our bots memories which help it keep track of events.

In [12]:
# Writing stories and saving it in the stories.md file

stories_md = """
##  Greeting path             
* greet              
  - utter_greet

##  business path             
* company_name              
  - utter_company_name

## team path
* team_info
  - utter_teammenu
* choosing_teamsmenu
  - utter_team_info

## males Path
* males_info
  - utter_malesmenu
* choosing_malessmenu
  - utter_males_info

## females Path
* females_info
  - utter_femalesmenu
* choosing_femalessmenu
  - utter_females_info

## businessage Path
* businessage
  - utter_businessagemenu
* choosing_businessagesmenu
  - utter_businessage_info

## teamsuccess Path
* team_success
  - utter_teamsuccess
* choosing_team_successsmenu
  - utter_team_info

## teamdiversity Path
* team_diversity
  - utter_teamdiversity
* choosing_team_diversitysmenu
  - utter_team_info

## teaminclusiveness Path
* team_inclusiveness
  - utter_teaminclusiveness
* choosing_team_inclusivenesssmenu
  - utter_team_info

## teamdiversitysuccess Path
* team_diversitysuccess
  - utter_teamdiversitysuccess
* choosing_team_diversitysuccesssmenu
  - utter_team_info

## teamcountry Path
* team_country
  - utter_teamcountry

## teamiswomenowned Path
* iswomenowned
  - utter_iswomenowned
* choosing_team_iswomenownedmenu
  - utter_team_info

## teamminiorityowned Path
* team_miniorityowned
  - utter_teamminiorityowned
* choosing_team_miniorityownedsmenu
  - utter_team_info

## thanks path
* thanks
  - utter_thanks  

## say goodbye
* goodbye
  - utter_goodbye

## thanks
* thanks
  - utter_thanks

## fallback rule
* bot_fallback
  - action_default_fallback

"""

%store stories_md >stories.md

Writing 'stories_md' (str) to file 'stories.md'.


The domain is where we link together all our files for the chatbot. The uttar_default action is he action to be taken when the confidence value of all possible intentions is below the thereshold value set inn our configuration file under the fallback policy.

In [13]:
# Writing all the intents,slots,entities,actions and templates to domain.yml

domain_yml = """
intents:
- greet
- goodbye
- company_name
- team_info
- choosing_team
- males_info
- choosing_males
- females_info
- choosing_females
- businessage
- choosing_businessage
- team_success
- choosing_teamsuccess
- team_diversity
- choosing_teamdiversity
- team_inclusiveness
- choosing_teaminclusiveness
- team_diversitysuccess
- choosing_teamdiversitysuccess
- team_country
- iswomenowned
- choosing_iswomenowned
- team_miniorityowned
- choosing_teamminiorityowned
- thanks



slots:
  group:
    type: text

entities:
- group


actions:
- utter_greet
- utter_company_name
- utter_teammenu
- utter_team_info
- utter_malesmenu
- utter_males_info
- utter_femalesmenu
- utter_females_info
- utter_businessagemenu
- utter_businessage_info
- utter_teamsuccess
- utter_team_success_info
- utter_teamdiversity
- utter_team_diversity_info
- utter_teaminclusiveness
- utter_team_inclusiveness_info
- utter_teamdiversitysuccess
- utter_team_diversitysuccess_info
- utter_teamcountry
- utter_iswomenowned
- utter_team_iswomenowned_info
- utter_teamminiorityowned
- utter_team_miniorityowned_info
- utter_goodbye
- utter_thanks
- utter_default


templates:
  utter_greet:
  - text: "\n\n Perceptron BOT: \n\n Hey! Welcome to Perceptron terminal . How may I help you ? \n\n \n\n You:"

  utter_company_name:
  - text: "\n\n Perceptron BOT: \n\n Please provide the name of your business. \n\n \n\n  You:"

  utter_teammenu:
  - text: "\n\n Perceptron BOT: \n\n Please choose the team strength from the menu option: \n\n 1. 5  2. 10 3. 20  4. 50. \n\n Enter the appropraite menu option based on your team strength. \n\n \n\n  You:"

  utter_team_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_malesmenu:
  - text: "\n\n Perceptron BOT: \n\n Please choose the number of male team members in your team from the menu option: \n\n 1. 10  2. 20 3. 30  4. 40 \n\n Enter the appropraite menu option. \n\n \n\n  You:"

  utter_males_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_femalesmenu:
  - text: "\n\n Perceptron BOT: \n\n Please choose the number of female team members in your team from the menu option: \n\n 1. 10  2. 20 3. 30  4. 40 \n\n Enter the appropraite menu option. \n\n \n\n  You:"

  utter_females_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_businessagemenu:
  - text: "\n\n Perceptron BOT: \n\n Please choose the range of business age of your team from the menu option: \n\n 1. 10-20  2. 20-30 3. 30-40  4. 40-50 \n\n Enter the appropraite menu option. \n\n \n\n  You:"

  utter_businessage_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_teamsuccess:
  - text: "\n\n Perceptron BOT: \n\n Please rate the success of your team from the menu option: \n\n 1. 1  2. 2 3. 3  4. 4 5. 5 \n\n Enter the appropraite menu option. \n\n \n\n  You:"

  utter_team_success_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_teamdiversity:
  - text: "\n\n Perceptron BOT: \n\n Please rate the diversity of your team from the menu option: \n\n 1. 1  2. 2 3. 3  4. 4 5. 5 \n\n Enter the appropraite menu option. \n\n \n\n  You:"

  utter_team_diversity_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_teaminclusiveness:
  - text: "\n\n Perceptron BOT: \n\n Please rate the inclusiveness of your team from the menu option: \n\n 1. 1  2. 2 3. 3  4. 4 5. 5 \n\n Enter the appropraite menu option. \n\n \n\n  You:"

  utter_team_inclusiveness_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_teamdiversitysuccess:
  - text: "\n\n Perceptron BOT: \n\n Please rate the diversity success of your team from the menu option: \n\n 1. 1  2. 2 3. 3  4. 4 5. 5 \n\n Enter the appropraite menu option. \n\n \n\n  You:"

  utter_team_diversitysuccess_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_teamcountry:
  - text: "\n\n Perceptron BOT: \n\n Please provide the country of your team \n\n \n\n  You:"

  utter_iswomenowned:
  - text: "\n\n Perceptron BOT: \n\n Please let us know if your team is women owned, if 1 then YES if 0 then NO: \n\n 1. 1  2. 0 \n\n Enter the appropraite menu option. \n\n \n\n  You:"

  utter_team_iswomenowned_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_teamminiorityowned:
  - text: "\n\n Perceptron BOT: \n\n Please let us know if your team is minority owned from the menu option: \n\n 1. Latinos 2. African_American 3. Asian 4. Female_total 5. Asian_female 6. African_American_female 7. Hispanic_or_Latino_female 8. Women_of_color 9. White_female 10. People_of_color 11. White 12. Native Americans \n\n Enter the appropraite menu option. \n\n \n\n  You:"

  utter_team_miniorityowned_info:
  - text: "\n\n Perceptron BOT: \n\n Thank you! Your response has been received. \n\n \n\n You:"

  utter_goodbye:
  - text: "\n\n Perceptron BOT: \n\n Thank you for choosing to transit with us. Waiting for your next visit. \n\n \n\n You:"
  
  utter_thanks:
  - text: "\n\n Perceptron BOT: \n\n my pleasure \n\n \n\n You:"

  utter_default:
  - text: "\n\n Perceptron BOT: \n\n I'm sorry, I didn't quite understand that. Could you rephrase? \n\n You:"

 
 
"""

%store domain_yml >domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


Here we import our policies and the Agent module. The Agent module is used to load the file, and passed to the 'agent' model. The 'agent' model is in turn trained with the data in the stories file.

In [14]:
# Import the policies and agent
from rasa_core.policies import FallbackPolicy, MemoizationPolicy,KerasPolicy
from rasa_core.agent import Agent

# Initialize the model with `domain.yml`
agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), ])

# loading our  training dialogues from `stories.md` 
training_data = agent.load_data('stories.md')


# Training the model
agent.train(
    training_data)
    #validation_split=0.0,
    #epochs=200)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 17/17 [00:00<00:00, 72.41it/s, # trackers=20]
Processed actions: 53it [00:00, 421.58it/s, # examples=53]


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 61)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                12032     
_________________________________________________________________
dense (Dense)                (None, 34)                1122      
_________________________________________________________________
activation (Activation)      (None, 34)                0         
Total params: 13,154
Trainable params: 13,154
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
607/607 [==============

Install version 2.10.0 of h5py package

In [15]:
!pip uninstall h5py
!pip install h5py==2.10.0

Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h5py-2.10.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-2.10.0
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)


Load the trained model

In [16]:
#Starting the Bot
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Write a function to tale inputs for the chatbot and gives out an output while stopping when 'stop' is typed in.

In [17]:
print("Your bot is ready to talk! Type your messages here or send 'stop'" + "\n\n" + "You:")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'

You:
Hi

Perceptron BOT:
Hey! Welcome to Perceptron terminal . How may I help you ?

You:
Company

Perceptron BOT:
Please provide the name of your business.

You:
Wells Fargo

Perceptron BOT:
Hey! Welcome to Perceptron terminal . How may I help you ?

You:
males

Perceptron BOT:
Please choose the number of male team members in your team from the menu option:
1. 10  2. 20 3. 30  4. 40
Enter the appropraite menu option.

You:
1

Perceptron BOT:
Thank you! Your response has been received.

You:
females

Perceptron BOT:
Please choose the number of female team members in your team from the menu option:
1. 10  2. 20 3. 30  4. 40
Enter the appropraite menu option.

You:
1

Perceptron BOT:
Thank you! Your response has been received.

You:
team strength

Perceptron BOT:
Please choose the team strength from the menu option:
1. 5  2. 10 3. 20  4. 50.
Enter the appropraite menu option based on your team strength.

You:
3

Perceptro